Import cv2img.py for Image preprocessing

In [36]:
import cv2img

In [37]:
cv2img.mains(imgin="certs\\10.jpeg", imgout="cert1")

DONE!


Import AzureReadAPI.py a Read API wrapper!

In [1]:
import AzureReadAPI

In [2]:
test = AzureReadAPI.mains(img_path="certs/16.jpeg")

In [3]:
test

'27 -29 March Diginique Techlobe 2020 IIT ROORKEE |COGNIZANCE An IIT Roorkee Alumni Venture Dated: 28"- 20" feb 2020 Venue: MIT-WPU PUNe Certificate of Participation This is to certify that Mr. Alphons Jaimon has attended the two days workshop on "Machine Learning & Artificial Intelligence" conducted by Diginique TechLabs in association with Cognizance\'20 IIT Roorkee. Sudhir Chaudhary Alok Birthare CEO Overall Coordinator Diginique TechLabs Cognizance\' 20, HITR'

### NLTK + Spacy!
We dont need cv2img for this as Azure Cognitive Service Read API uses a whole different methed for text extraction better use a HIGH resolution pic but less than 20mb!